Transfer Learning using VGG16

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Dropout
from keras.models import Sequential, Model

def df_detection_model_crnn(inp_shape):
    # using transfer learning from vgg16
    vgg = VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
    
    for layer in vgg.layers:
        layer.trainable = True
    
    model = Sequential()
    
    
    model.add(TimeDistributed(vgg.layers[0],input_shape=inp_shape))
    
    for i in range(1,len(vgg.layers)):
        model.add(TimeDistributed(vgg.layers[i]))

        
    #model.add(TimeDistributed(vgg,input_shape=inp_shape))  # (5,224,224,3) -> (5,7,7,512)
    model.add(TimeDistributed(Flatten()))   # (5,7,7,512) -> (512*7*7*5)
    model.add(LSTM(512, activation='tanh',return_sequences=True))
    model.add(LSTM(256, activation='tanh'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(2,activation='softmax'))
    model.summary()
    return model

Transfer Learning using VGG-16 BiLSTM

In [ ]:
import tensorflow as tf
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Dropout, Bidirectional
from keras.models import Sequential, Model

def df_detection_model_crnn(inp_shape):
    # using transfer learning from vgg16
    vgg = VGG16(weights='imagenet',include_top=False, input_shape=(224,224,3))
    
    for layer in vgg.layers:
        layer.trainable = True
    
    model = Sequential()
    
    
    model.add(TimeDistributed(vgg.layers[0],input_shape=inp_shape))
    
    for i in range(1,len(vgg.layers)):
        model.add(TimeDistributed(vgg.layers[i]))

        
    #model.add(TimeDistributed(vgg,input_shape=inp_shape))  # (30,224,224,3) -> (30,7,7,512)
    model.add(TimeDistributed(Flatten()))   # (30,7,7,512) -> (512)
    model.add(Bidirectional(LSTM(512, activation='tanh',return_sequences=True)))
    model.add(Bidirectional(LSTM(256, activation='tanh')))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(2,activation='softmax'))
    model.summary()
    return model

Transfer Learning using VGG-19

In [ ]:
import tensorflow as tf
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Dropout
from keras.models import Sequential, Model

def df_detection_model_crnn(inp_shape):
    # using transfer learning from vgg19
    vgg = VGG19(weights='imagenet',include_top=False, input_shape=(224,224,3))
    
    for layer in vgg.layers:
        layer.trainable = True
    
    model = Sequential()
    
    
    model.add(TimeDistributed(vgg.layers[0],input_shape=inp_shape))
    
    for i in range(1,len(vgg.layers)):
        model.add(TimeDistributed(vgg.layers[i]))

        
    #model.add(TimeDistributed(vgg,input_shape=inp_shape))  # (30,224,224,3) -> (30,7,7,512)
    model.add(TimeDistributed(Flatten()))   # (30,7,7,512) -> (512)
    model.add(LSTM(512, activation='tanh',return_sequences=True))
    model.add(LSTM(256, activation='tanh'))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(2,activation='softmax'))
    model.summary()
    return model

Transfer Learning using VGG19 BiLSTM

In [ ]:
import tensorflow as tf
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, LSTM, Flatten, TimeDistributed, Dropout, Bidirectional
from keras.models import Sequential, Model

def df_detection_model_crnn(inp_shape):
    # using transfer learning from vgg19
    vgg = VGG19(weights='imagenet',include_top=False, input_shape=(224,224,3))
    
    for layer in vgg.layers:
        layer.trainable = True
    
    model = Sequential()
    
    
    model.add(TimeDistributed(vgg.layers[0],input_shape=inp_shape))
    
    for i in range(1,len(vgg.layers)):
        model.add(TimeDistributed(vgg.layers[i]))

        
    #model.add(TimeDistributed(vgg,input_shape=inp_shape))  # (30,224,224,3) -> (30,7,7,512)
    model.add(TimeDistributed(Flatten()))   # (30,7,7,512) -> (512)
    model.add(Bidirectional(LSTM(512, activation='tanh',return_sequences=True)))
    model.add(Bidirectional(LSTM(256, activation='tanh')))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(32,activation='relu'))
    model.add(Dropout(.3))
    model.add(Dense(2,activation='softmax'))
    model.summary()
    return model

In [ ]:
%tensorboard --logdir=logs/

In [ ]:
!pip install keras-video-generators
import glob
import numpy as np
import keras
import tensorflow as tf
import psutil
from keras_video import VideoFrameGenerator
import keras_video.utils
import os
import matplotlib.pyplot as plt
from time import time
from keras.callbacks import TensorBoard

classes = [i.split(os.path.sep)[4] for i in glob.glob('../input/deepfake-dataset/Celeb-DF-v2/*')]
classes.sort()
print(classes)

TSHAPE = (224,224)
CHANNELS = 3
NBFRAME = 5
BS = 10
# pattern to get videos and classes
glob_pattern='../input/deepfake-dataset/Celeb-DF-v2/{classname}/*.mp4'

train = VideoFrameGenerator(
    classes=classes, 
    glob_pattern=glob_pattern,
    nb_frames=NBFRAME,
    split_val=.1,
    split_test=.4,
    shuffle=True,
    batch_size=BS,
    target_shape=TSHAPE,
    nb_channel=CHANNELS,
    use_frame_cache=False)

valid = train.get_validation_generator()

test = train.get_test_generator()

keras_video.utils.show_sample(train)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [ ]:
INPSHAPE = (NBFRAME,224,224,3)

model = df_detection_model_crnn(INPSHAPE)
#model.load_weights('../input/weights-batch-3/weights_batch3.hdf5')

METRICS = [
    keras.metrics.CategoricalAccuracy(name='categorical_accuracy')
]

optimizer = keras.optimizers.SGD(lr = 0.001)
loss = keras.losses.CategoricalCrossentropy()
model.compile(
    optimizer,
    loss=loss,
    metrics=METRICS
)

EPOCHS=100

# callbacks = [
#     keras.callbacks.ReduceLROnPlateau(verbose=1),
#     keras.callbacks.ModelCheckpoint(
#         '/kaggle/working/weights_batch4.hdf5',
#         verbose=1,save_best_only=True),
# ]

#tensorboard_callback = TensorBoard(log_dir='logs/{}'.format(time()))

history = model.fit_generator(
    train,
    validation_data = valid,
    verbose=1,
    epochs=EPOCHS,
    shuffle=True
)

print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
output = model.evaluate_generator(test,verbose=1)
print(output)

In [ ]:
model.save('/kaggle/working/final_model_vgg16_bilstm.h5')